# CLIP: Localized Sea Level Rise

This python notebook downloads the necessary data for visualizing sea level rise (temporal extent 2002-01 -2019-12) from this URL (https://www.seanoe.org/data/00631/74354/) and converts it into a format that is usable for Datawrapper. This includes an aggregation of the measurement values and a temporal filtering for July 2019.

It requires standard modules like os, and shutil. Modules like requests, pandas and xarray might have to be installed depending on local setup & environment.

## Imports

In [1]:
import os
import shutil
import requests
import pandas as pd
import xarray as xr

## Data Download

In [2]:
url = 'https://www.seanoe.org/data/00631/74354/data/98856.zip'

req = requests.get(url)
filename = 'sla_data.zip'

with open(filename,'wb') as output_file:
    output_file.write(req.content)

In [3]:
unzip_path = "nc_files"
isExist = os.path.exists(unzip_path)
if not isExist:
   os.makedirs(unzip_path)

In [4]:
shutil.unpack_archive('sla_data.zip', unzip_path)

## Data Processing

In [5]:
#function to extract data from .nc files per station and append to value list
def nc2csv(infile, year, month, list_viz):
  ds = xr.open_dataset(infile)
  ds["year"] = ds["time"].dt.strftime("%Y")
  ds["month"] = ds["time"].dt.strftime("%m")

  df = ds.to_dataframe().reset_index()
  df = df[['lat', 'lon', 'year', 'month', 'local_sla_trend']].copy()

  df = df.loc[(df['year'] == year) & (df['month'] == month)]
  df = df[['lat', 'lon', 'local_sla_trend']].copy()

  #aggregate submeasurements
  df = df.agg(['mean'])

  values = df.values.tolist()
  list_viz.append(values[0])

In [6]:
#iterate over nc files and extract one datapoint per station
list_viz = []
for root, _, files in os.walk(unzip_path):
    for x in files:
        if x.endswith('.nc'):
          print(x)
          x = 'nc_files/' + x
          nc2csv(x, '2019', '07', list_viz)

ESACCI-SEALEVEL-IND-MSLTR-MERGED-WAFRICA_JA_174_05-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_064_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-NW_AMERICA_JA_245_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_051_04-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-HUMBOLDT_JA_028_09-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-S_AUSTRALIA_JA_140_03-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-WAFRICA_JA_198_02-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-MED_SEA_JA_059_05-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-NW_AMERICA_JA_095_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_205_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_140_06-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-NW_AMERICA_JA_078_04-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_151_08-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-N_INDIAN_JA_090_03-20221220-fv02.2.nc
ESACC

## Data Export for Visualization

In [7]:
# create datafrane for export
df = pd.DataFrame (list_viz, columns = ['lat', 'lng', 'sla'])

# filter European stations
df = df.loc[(df['lat'] > 30.0) & (df['lat'] < 64.0)]
df = df.loc[(df['lng'] > -30.0) & (df['lng'] < 70.0)]

# round coords for smaller file size
df = df.round(3)

# save as csv
df.to_csv('viz_sla.csv', index=False)

In [8]:
df.head()

,lat,lng,sla
6,34.314,-6.779,2.453
7,36.993,14.104,2.740
21,48.844,-4.062,3.196
27,36.634,31.484,2.383
30,31.563,31.626,2.630
